### Dataset Preparation

Installing

In [ ]:
'''
!sudo echo -ne '\n' | sudo add-apt-repository ppa:alessandro-strada/ppa >/dev/null 2>&1 # note: >/dev/null 2>&1 is used to supress printing
!sudo apt update >/dev/null 2>&1
!sudo apt install google-drive-ocamlfuse >/dev/null 2>&1
!google-drive-ocamlfuse
!sudo apt-get install w3m >/dev/null 2>&1 # to act as web browser
!xdg-settings set default-web-browser w3m.desktop >/dev/null 2>&1 # to set default browser
%cd /content
!mkdir gdrive
%cd gdrive
!mkdir "My Drive"
!google-drive-ocamlfuse "/content/gdrive/My Drive"
'''

In [ ]:
#%%capture
!pip install bitsandbytes
!pip install transformers==4.34.0
!pip install -U peft
!pip install accelerate
!pip install -U trl
!pip install datasets==2.16.0
!pip install wandb

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
from peft import PeftModel
import torch

In [ ]:
from google.colab import userdata
HUGGING_FACE_HUB_TOKEN=userdata.get('HUGGING_FACE_HUB_TOKEN')
secret_wandb=userdata.get('wandb')

In [ ]:
!huggingface-cli login --token $secret_hf

In [ ]:
base_model = "mistralai/Mistral-7B-v0.1"
new_model = "DataIntelligenceTeam/mistral_7B_NER"

### Merging the adopter

### Load base model with adopter

In [ ]:
base_model_reload = AutoModelForCausalLM.from_pretrained(
    base_model, low_cpu_mem_usage=True,
    return_dict=True,torch_dtype=torch.bfloat16,
    device_map= "auto")

model = PeftModel.from_pretrained(base_model_reload, new_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1363: UserWarning: Current model requires 520101632 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


### Reload tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Inference

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartForCausalLM', 'Prophe

In [ ]:
prompt = """
<s>[INST]Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_volume by summing the volumes of all packages.
* quantity: Individual Quantity of a specific item being shipped.
* package_type: Individual Type of packaging used (e.g., pallets, cartons).
* weight: Individual Weight of a specific package.
* measures: Individual Dimensions or measurements of a package.
* stackable: Indicates whether the shipment is stackable (True or False).
* volume: Individual Volume of a specific package.
* commodity: Type of goods or commodities being shipped.
* company: Name of the email sending company, also the shipping company or carrier.
* incoterms: Choose available options: EXW, FCA, FAS, FOB, CFR, CIF, CPT, CIP, DAP, DPU, DDP.

For attributes with multiple values, such as measures, volume, weight, package_type, and quantity, provide each value separately in a JSON format.

### Input data:
Buongiorno, ; Ci servirebbe cortesemente Vs quotazione per: ; PESO:  7140 ; COLLI:  7 Pallet ; MISURE: 100X120X130H (6 PLTS)   100X120X160H (1 PLT) ; La dogana verrà fatta a stabilimento, pertanto si richiede solo lo scarico bolla. ; La attendiamo ; Grazie mille ; Nicolo Esibiti ; Export Sales Department ; T +39 010 2717399 ; Casasco & Nardi S.p.A. ; Palazzo Andrea Pitto ; Piazza Fossatello, 3 ; 16124, Genova ; www.casasconardi.com ;

### Output:

"""

In [ ]:
sequences = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=100,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] 
<s>[INST]Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate t

### Pushing the fine-tuned model

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
model = model.merge_and_unload()

In [ ]:
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DataIntelligenceTeam/mistral_7B_NER/commit/2f2b7165bef45b86f5aaed0d5a098db46c75c160', commit_message='Upload tokenizer', commit_description='', oid='2f2b7165bef45b86f5aaed0d5a098db46c75c160', pr_url=None, pr_revision=None, pr_num=None)

### Accessing the fine-tuned model

In [ ]:
from transformers import pipeline


pipe = pipeline(
    "text-generation",
    model = "DataIntelligenceTeam/mistral_7B_NER",
    device_map="auto"
)

sequences = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=200,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,)

print(sequences[0]['generated_text'])